In [1]:
import json

journals_path = r"C:\Users\Manaswini\Downloads\ashwam_stability\data\journals.jsonl"

journals = []
with open(journals_path, "r", encoding="utf-8") as f:
    for line in f:
        journals.append(json.loads(line))

len(journals)


5

In [2]:
journals

[{'journal_id': 'B001',
  'created_at': '2025-12-10T21:35:00+11:00',
  'text': 'Woke up okay but by afternoon I felt this dull headache behind my eyes. Had masala chai with 2 tsp sugar. Lunch was rice + dal + achar. Emotionally a bit edgy, snapping at people for no reason. By evening, mind felt scattered—kept forgetting what I opened the laptop for.',
  'lang_hint': 'en'},
 {'journal_id': 'B002',
  'created_at': '2025-12-11T07:50:00+11:00',
  'text': 'Kal raat neend theek nahi thi. 3am ko uthi, phir 30 min tak aankh hi nahi lagi. Subah coffee pi and skipped breakfast. Feeling anxious in chest, like tightness. No cramps today though.',
  'lang_hint': 'hinglish'},
 {'journal_id': 'B003',
  'created_at': '2025-12-11T19:10:00+11:00',
  'text': 'Dinner: paneer bhurji + 2 rotis. After eating, I got super sleepy and my stomach felt bloated. Mood was actually good—felt calm and grateful. Brain felt clear, focused while reading.',
  'lang_hint': 'en'},
 {'journal_id': 'B004',
  'created_at': '2

In [3]:
import os

runs_base = r"C:\Users\Manaswini\Downloads\ashwam_stability\data\llm_runs"
files = sorted(os.listdir(runs_base))
files


['B001.run1.json',
 'B001.run2.json',
 'B001.run3.json',
 'B002.run1.json',
 'B002.run2.json',
 'B002.run3.json',
 'B003.run1.json',
 'B003.run2.json',
 'B003.run3.json',
 'B004.run1.json',
 'B004.run2.json',
 'B004.run3.json',
 'B005.run1.json',
 'B005.run2.json',
 'B005.run3.json']

In [4]:
from collections import defaultdict

runs_by_journal = defaultdict(list)

for fname in files:
    journal_id = fname.split(".")[0]   # B001
    with open(os.path.join(runs_base, fname), "r", encoding="utf-8") as f:
        runs_by_journal[journal_id].append(json.load(f))

# check
{k: len(v) for k, v in runs_by_journal.items()}


{'B001': 3, 'B002': 3, 'B003': 3, 'B004': 3, 'B005': 3}

In [5]:
runs_by_journal["B001"]


[{'journal_id': 'B001',
  'run_id': 'run1',
  'items': [{'domain': 'symptom',
    'text': 'dull headache',
    'evidence_span': 'dull headache behind my eyes',
    'polarity': 'present',
    'intensity_bucket': 'medium',
    'time_bucket': 'today',
    'confidence': 0.78},
   {'domain': 'food',
    'text': 'masala chai (2 tsp sugar)',
    'evidence_span': 'masala chai with 2 tsp sugar',
    'polarity': 'present',
    'intensity_bucket': 'unknown',
    'time_bucket': 'today',
    'confidence': 0.92},
   {'domain': 'food',
    'text': 'rice + dal + achar',
    'evidence_span': 'Lunch was rice + dal + achar',
    'polarity': 'present',
    'intensity_bucket': 'unknown',
    'time_bucket': 'today',
    'confidence': 0.9},
   {'domain': 'emotion',
    'text': 'edgy / snappy',
    'evidence_span': 'a bit edgy, snapping at people for no reason',
    'polarity': 'present',
    'arousal_bucket': 'high',
    'time_bucket': 'today',
    'confidence': 0.72},
   {'domain': 'mind',
    'text': 'scat

In [7]:
for i, run in enumerate(runs_by_journal["B001"], 1):
    print(f"\n--- RUN {i} ---")
    for obj in run["items"]:
        print(obj)


--- RUN 1 ---
{'domain': 'symptom', 'text': 'dull headache', 'evidence_span': 'dull headache behind my eyes', 'polarity': 'present', 'intensity_bucket': 'medium', 'time_bucket': 'today', 'confidence': 0.78}
{'domain': 'food', 'text': 'masala chai (2 tsp sugar)', 'evidence_span': 'masala chai with 2 tsp sugar', 'polarity': 'present', 'intensity_bucket': 'unknown', 'time_bucket': 'today', 'confidence': 0.92}
{'domain': 'food', 'text': 'rice + dal + achar', 'evidence_span': 'Lunch was rice + dal + achar', 'polarity': 'present', 'intensity_bucket': 'unknown', 'time_bucket': 'today', 'confidence': 0.9}
{'domain': 'emotion', 'text': 'edgy / snappy', 'evidence_span': 'a bit edgy, snapping at people for no reason', 'polarity': 'present', 'arousal_bucket': 'high', 'time_bucket': 'today', 'confidence': 0.72}
{'domain': 'mind', 'text': 'scattered / forgetful', 'evidence_span': 'mind felt scattered—kept forgetting what I opened the laptop for', 'polarity': 'present', 'intensity_bucket': 'medium',

In [8]:
def evidence_overlap(a: str, b: str) -> float:
    """
    Token-level Jaccard overlap
    """
    a_tokens = set(a.lower().split())
    b_tokens = set(b.lower().split())
    if not a_tokens or not b_tokens:
        return 0.0
    return len(a_tokens & b_tokens) / len(a_tokens | b_tokens)


In [9]:
def match_items(run_a, run_b, overlap_threshold=0.8):
    """
    Match items between two runs using:
    - same domain
    - evidence span overlap
    Returns list of (item_a, item_b) matches
    """
    matches = []
    used_b = set()

    for i, item_a in enumerate(run_a["items"]):
        best_j = None
        best_score = 0.0

        for j, item_b in enumerate(run_b["items"]):
            if j in used_b:
                continue
            if item_a["domain"] != item_b["domain"]:
                continue

            score = evidence_overlap(
                item_a["evidence_span"],
                item_b["evidence_span"]
            )

            if score > best_score:
                best_score = score
                best_j = j

        if best_j is not None and best_score >= overlap_threshold:
            used_b.add(best_j)
            matches.append((item_a, run_b["items"][best_j]))

    return matches


In [10]:
run1, run2, run3 = runs_by_journal["B001"]

matches_12 = match_items(run1, run2)
matches_13 = match_items(run1, run3)

len(matches_12), len(matches_13)


(4, 5)

In [11]:
for a, b in matches_13:
    print(a["domain"], " | ", a["polarity"], "→", b["polarity"])


symptom  |  present → present
food  |  present → present
food  |  present → present
emotion  |  present → present
mind  |  present → present


In [12]:
BUCKET_FIELDS = ["intensity_bucket", "arousal_bucket", "time_bucket"]

def bucket_drift_rate(matches):
    drift = 0
    total = 0

    for a, b in matches:
        for field in BUCKET_FIELDS:
            if field in a or field in b:
                total += 1
                if a.get(field) != b.get(field):
                    drift += 1

    return drift / total if total > 0 else 0.0


In [13]:
matches_12 = match_items(runs_by_journal["B001"][0], runs_by_journal["B001"][1])
matches_13 = match_items(runs_by_journal["B001"][0], runs_by_journal["B001"][2])

print("Run1 vs Run2 drift:", bucket_drift_rate(matches_12))
print("Run1 vs Run3 drift:", bucket_drift_rate(matches_13))


Run1 vs Run2 drift: 0.0
Run1 vs Run3 drift: 0.3


In [14]:
def agreement_rate(run_a, run_b):
    matches = match_items(run_a, run_b)
    matched = len(matches)
    total = len(run_a["items"]) + len(run_b["items"]) - matched
    return matched / total if total > 0 else 0.0


In [15]:
print("Agreement Run1 vs Run2:", agreement_rate(
    runs_by_journal["B001"][0],
    runs_by_journal["B001"][1]
))

print("Agreement Run1 vs Run3:", agreement_rate(
    runs_by_journal["B001"][0],
    runs_by_journal["B001"][2]
))


Agreement Run1 vs Run2: 0.8
Agreement Run1 vs Run3: 1.0


In [16]:
from collections import Counter

def majority_vote(values, allow_uncertain=True):
    """
    Take majority from a list of values.
    If tie or all missing and allow_uncertain=True → return 'uncertain'
    """
    count = Counter(values)
    if not count:
        return 'uncertain' if allow_uncertain else None
    most_common, freq = count.most_common(1)[0]
    if list(count.values()).count(freq) > 1 and allow_uncertain:
        return 'uncertain'
    return most_common


In [17]:
def merge_runs(run1, run2, run3, overlap_threshold=0.8):
    runs = [run1, run2, run3]
    all_items = []
    for r in runs:
        for item in r["items"]:
            all_items.append(item)
    
    grouped = {}
    for item in all_items:
        key = (item["domain"], item["evidence_span"])
        if key not in grouped:
            grouped[key] = []
        grouped[key].append(item)
    
    merged_items = []
    for key, objs in grouped.items():
        merged = {}
        merged["domain"] = key[0]
        merged["evidence_span"] = key[1]
        # Fields to take majority vote
        for field in ["polarity", "time_bucket", "intensity_bucket", "arousal_bucket"]:
            values = [o.get(field) for o in objs if o.get(field) is not None]
            merged[field] = majority_vote(values)
        merged_items.append(merged)
    
    return merged_items


In [18]:
stable_B001 = merge_runs(
    runs_by_journal["B001"][0],
    runs_by_journal["B001"][1],
    runs_by_journal["B001"][2]
)

for obj in stable_B001:
    print(obj)


{'domain': 'symptom', 'evidence_span': 'dull headache behind my eyes', 'polarity': 'present', 'time_bucket': 'today', 'intensity_bucket': 'medium', 'arousal_bucket': 'uncertain'}
{'domain': 'food', 'evidence_span': 'masala chai with 2 tsp sugar', 'polarity': 'present', 'time_bucket': 'today', 'intensity_bucket': 'unknown', 'arousal_bucket': 'uncertain'}
{'domain': 'food', 'evidence_span': 'Lunch was rice + dal + achar', 'polarity': 'present', 'time_bucket': 'today', 'intensity_bucket': 'unknown', 'arousal_bucket': 'uncertain'}
{'domain': 'emotion', 'evidence_span': 'a bit edgy, snapping at people for no reason', 'polarity': 'present', 'time_bucket': 'today', 'intensity_bucket': 'uncertain', 'arousal_bucket': 'high'}
{'domain': 'mind', 'evidence_span': 'mind felt scattered—kept forgetting what I opened the laptop for', 'polarity': 'present', 'time_bucket': 'today', 'intensity_bucket': 'medium', 'arousal_bucket': 'uncertain'}


In [19]:
stable_all_journals = {}
for journal_id, runs in runs_by_journal.items():
    stable_all_journals[journal_id] = merge_runs(runs[0], runs[1], runs[2])


In [20]:
import json
import os

output_dir = r"C:\Users\Manaswini\Downloads\ashwam_stability\outputs"
os.makedirs(output_dir, exist_ok=True)

for journal_id, items in stable_all_journals.items():
    with open(os.path.join(output_dir, f"{journal_id}_stable.json"), "w") as f:
        json.dump(items, f, indent=2)

with open(os.path.join(output_dir, "stable_all_journals.json"), "w") as f:
    json.dump(stable_all_journals, f, indent=2)

print(f"✅ Stable outputs saved in {output_dir}")


✅ Stable outputs saved in C:\Users\Manaswini\Downloads\ashwam_stability\outputs
